# Part 4

### Problem Statement
Choose one of the gradient boosting implementations we covered in week 5
i.e. one of H2O, lightbgm or catboost. You should carefully research relevant
documents for the key parameters and how to tune them. Create your best
model and submit on Kaggle.

Marks will be awarded for work which:
- Demonstrates that the user documents for version of boosting used have been thoroughly read

- Includes tuning of all relevant hyper-parameters, and a summary of which h-p’s mattered most for the final result and why this might be the case.

- Shows efforts to use hyper-opt or other dedicated h-p training softwares.

- Uses custom loss functions which penalise overfitting.

- Is easy to follow and very well commented. In particular where comments help me to follow your thought process and demonstrate an understanding of what you are doing.

*Disclaimer: Some preprocessing processes were using the same code as of Part 2,3

## Set-up

### Import relevant libraries

In [1]:
import os
import numpy as np
import pandas as pd
import pickle

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder

from catboost import CatBoostClassifier, Pool, cv, EShapCalcType, EFeaturesSelectionAlgorithm
import hyperopt

In [2]:
# install ipywidgets package to appreciate the plotting feature for catboost
# !pip install ipywidgets
# !jupyter nbextension install --py widgetsnbextension --user
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


### Set directories and paths

In [3]:
# set directories
print(os.getcwd())
dirRawData = os.path.join('..', 'input')
dirPData   = os.path.join('..', 'PData')
dirPOutput = os.path.join('..', 'POutput')

/home/jovyan/smm284-aml/assignment/PCode


### Load data

In [4]:
# load data: 250k train data
f_name = '01_df_250k.pickle'
f_path = os.path.join(dirPData, f_name)
PData = pickle.load(open(f_path, "rb"))

# separate data into train/test set
train_set = PData['df_train']
test_set = PData['df_test']

In [5]:
# set directories
print(os.getcwd())
dirRawData = os.path.join('..', 'input')
dirPData   = os.path.join('..', 'PData')
dirPOutput = os.path.join('..', 'POutput')

/home/jovyan/smm284-aml/assignment/PCode


In [6]:
# load variable metadata 
f_name = '01_vars.pickle'
f_path = os.path.join(dirPData, f_name)
var_meta = pickle.load(open(f_path, "rb"))

# extract lists from metadata
var_idx_num = var_meta['vars_ind_numeric']
var_idx_cat = var_meta['vars_ind_categorical']
var_idx_hccv = var_meta['vars_ind_hccv']
var_idx_id = var_meta['vars_notToUse']
var_idx_response = var_meta['var_dep']

### Clean missing values
#### - Deal appropriately with missings (for all numeric variables, -99 means missing).

In [7]:
# replace -99 with np.nan
train_set = train_set.replace(-99, np.nan)
test_set = test_set.replace(-99, np.nan)

In [8]:
# count number of nulls in each column -- train_set
np.array(train_set.isnull().sum(axis = 0))

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0, 102984,
        23788,  23788,  23788,   1852,   2090,  23898,  24321,  11673,
        32806,  33764, 197803,   8895,   6151,  55544,  57869,  57136,
            2,      0,      0,      0,      0,      0,      0,   1092,
            0,      0,      0,      0,      0,      0,      0,      1,
            1,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0])

In [9]:
# count number of nulls in each column -- test_set
np.array(test_set.isnull().sum(axis = 0))

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0, 283601,
            0,      0,      0,      0,      0,      2,      0,      1,
            0,      0,      0,      0,      0,      1,      0,      9,
          226,  10870,      9,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,     17, 131970,  26261,
        26261,  26261,   3429,   3504,  26298,  26356,  18076,   2261,
         2356,   2756,  10130,   6363,  18816,  22520,  20432,      2,
            0,      0,      0,      0,      0,      0,   3560,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0])

Strategy:
- remove the feature if the number of null values in the `test_set` exceed 250
- drop rows that have leftover null values in the `train_set`
- for leftover null values in the `test_set` impute with `mean` for numerical features and `most frequent` for categorical features

Through data exploration, we decided to remove features that have over 250 null values in the test set. That is because imputation can introduce biases to the data as it is impossible to perfectly predict the true values that are missing. Still, it is not possible to remove observations from the test set concerning the competition. We want to preserve as much information as possible and thus, for features with fewer than 250 null values (less than 0.1% of the data) imputation for mean / most frequent for numeric / categorical features is incorporated as imputing small number of observations with these values does not have a large impact on the overall distribution.

For the train set, after removing features with over 250 null values in the test set, very small numbers of null values are observed (5 in total). We decided to remove those observations and thus we only lose 3 observations in total.

In [10]:
# number of null values in the test set exceed 250
to_remove = test_set.columns[np.array(test_set.isnull().sum(axis = 0) > 250)]

In [11]:
# remove the variables
train_set = train_set.drop(columns=to_remove)
test_set = test_set.drop(columns=to_remove)

In [12]:
# display number of leftover missing values in `train_set`
np.array(train_set.isnull().sum(axis = 0))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [13]:
# drop rows that have leftover null values in the `train_set`
train_set = train_set.dropna(axis=0)

In [14]:
# display number of leftover missing values in `test_set`
np.array(test_set.isnull().sum(axis = 0))

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   2,   0,   1,   0,   0,   0,
         0,   0,   1,   0,   9, 226,   9,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,  17,   2,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [15]:
# get columns in the test_set with missing values
impute_col = test_set.columns[np.array(test_set.isnull().sum(axis = 0) > 0)]
impute_col

Index(['c09', 'e03', 'e24', 'e17', 'e18', 'e20', 'f10', 'e02'], dtype='object')

In [16]:
# display data for columns that are going to impute
to_impute = test_set[impute_col]
to_impute.head()

,c09,e03,e24,e17,e18,e20,f10,e02
0,B,A,A,920BD,11250,A4893,CKG,6.0
1,B,A,A,F33BC,5614E,D53C5,AIR,15.0
2,B,A,A,F33BC,5614E,D53C5,AEL,15.0
3,A,A,A,861C8,9076B,A4893,CUO,27.0
4,A,A,A,861C8,9076B,A4893,AWV,27.0


In [17]:
# get column indices for numerical and categorical features that need imputation
num_col = to_impute.select_dtypes(include='number').columns
cat_col = to_impute.select_dtypes(include='category').columns
# create imputation pipeline
# -- parameters = List of (name, transformer, columns)
pipeline = ColumnTransformer([
    ('cat_impute', SimpleImputer(strategy='most_frequent'), cat_col),
    ('num_impute', SimpleImputer(strategy='mean'), num_col)
])
# impute missing values
imputed_test = pipeline.fit_transform(test_set)

In [18]:
# assign values back to the `test_set`
test_set[impute_col] = imputed_test

Check again the number of missing values

In [19]:
print(f'Number of missing values in `train_set`: {np.sum(np.sum(train_set.isna()))}')
print(f'Number of missing values in `test_set`: {np.sum(np.sum(test_set.isna()))}')

Number of missing values in `train_set`: 0
Number of missing values in `test_set`: 0


As some features were removed, we adjust the metadata

In [20]:
var_idx_num = list(set(var_idx_num) - set(to_remove))
var_idx_cat = list(set(var_idx_cat) - set(to_remove))
var_idx_hccv = list(set(var_idx_hccv) - set(to_remove))

There is also a feature with only 1 unique value: `'e16'`, we removed it from the data.

In [21]:
# column with only 1 unique value
train_set.columns[(train_set.nunique() == 1)]

Index(['e16'], dtype='object')

In [22]:
# remove from the dataframes
train_set = train_set.drop(columns='e16')
test_set = test_set.drop(columns='e16')

# update metadata
var_idx_num.remove('e16')

### Prepare data for training

In [23]:
# get index for all independent variables
ind_idx_all = var_idx_num + var_idx_cat

In [24]:
# split data into train/validation sets
X_train, X_val, y_train, y_val  = train_test_split(train_set[ind_idx_all], train_set[var_idx_response],
                                                   test_size=0.1, random_state=888)

### Train Gradient Boosting through Catboost
Before tuning any parameter, we have trained a plain-vanilla CatBoost (i.e., leaving everything as default using the whole dataset). The AUC score we got on the Kaggle test set for the plain-vanilla model is `0.87561`

#### Feature selection
Reference: https://catboost.ai/docs/concepts/python-reference_catboost_select_features.html
- Algorithm: `RecursiveByShapValues` -- An optimisation of SHAP values was deemed to be the most accurate method

We performed feature selection through the default setting of CatBoost with: 
- loss_function: `Logloss`
- eval_metric: `AUC`
- random_seed: `999`

<img src="../POutput/part4_feature_selection.png">

The figure above shows the process for feature selection. It demonstrates that, to minimise the `Logloss`, 53 features should be removed.

The selected features are: `['e02', 'f02', 'f13', 'e23', 'a03', 'e11', 'f03', 'e13', 'f29', 'e14', 'e20', 'b04', 'f10', 'e21', 'e18', 'a01', 'f04', 'f20', 'e04', 'e09', 'a02']`

However, the performance on the test set is worse than the model trained with all the variables (`AUC = 0.87099` dropped by about 0.05). Therefore, we will fit our final model through the whole data set. We expect hyperparameter tuning would address the overfitting problem instead of selecting features before training. Also, the feature selection through a tree-based model seems to be more useful when selecting features to use in other models (Similar to how we used tree-based model to select important categorical features in Part 2).

In [ ]:
# # uncomment this cell if you want to perform feature selection
# # setup for feature selection
# params = {
#     'loss_function': 'Logloss',
#     'eval_metric': 'AUC',
#     'random_seed': 999,
# }

# # model specification
# model = CatBoostClassifier(**params)

# # feature selection
# summary = model.select_features(
#     train_pool,
#     eval_set=validate_pool,
#     features_for_select=ind_idx_all,
#     num_features_to_select=15, # let's say we want at least 15 features
#     steps=5, # number of times trained -- smaller number of features remove in each step
#     algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
#     shap_calc_type=EShapCalcType.Regular,
#     train_final_model=False,
#     logging_level='Verbose',
#     plot=True
# )

In [29]:
# # not used due to inferior performance
# # from the above selection we have
# selected_features = ['e02', 'f02', 'f13', 'e23', 'a03', 'e11', 'f03', 
#                      'e13', 'f29', 'e14', 'e20', 'b04', 'f10', 'e21', 
#                      'e18', 'a01', 'f04', 'f20', 'e04', 'e09', 'a02']
# # here are the 53 features we will not use in the model training process
# ignored_features = list(set(ind_idx_all) - set(selected_features))
# print(f'Number of features ignored: {len(ignored_features)}')

Number of features ignored: 53


#### Model hyperparameter specification:
Common Parameters:
- iteration: `1000` - number of trees to grow; default = 1000 
- loss_function: `Logloss` a.k.a. `Binary CrossEntropy` is used as an optimisation metric since our response variable is a binary class variable.
- eval_metric: `AUC` - the evaluation metric for this Kaggle competition.
- learning_rate: `default` - automatically selected by the package; the automatic setup is close to the optimal for a specific number of  iterations
- random_seed: `999` - for reproducibility
- l2_leaf_reg: `3.0` - L2 regularization; default = 3.0
- bootstrap_type: `Bernoulli` - tried `Bayesian` and `Bernoulli`
- random_strength: `1` - the amount of randomness during tree splits - reduce overfitting; default = 1
- use_best_model: `default` - `True` if validation exist; else: `False`
- depth: `default` - depth of the tree; interger up to 16
- grow_policy - `SymmetricTree` - tried `SymmetricTree`, `Depthwise`, and `Lossguide`; Symmetric tree was deemed to be the fastest and usually perform better than the others.
- ignored_features: `ignored_features` - use when ignoring features from the feature selection process
- boosting_type: `Plain` - since we train the model using CPU; only `Plain` is available

Overfitting detection settings:
- od_type: `Iter` - use `Iter` type with od_wait for earlystopping; though early_stopping_rounds parameter is also available and would do the same thing
- od_wait: `50` - number of iterations training after the optimal metric value

Performance settings:
- thread_count: `-1` - use all available cores; default = -1

Output settings:
- logging_level: `Verbose` - change to `Silence` if not care about log

#### Perform manual search on categorical kind of hyperparameters to see a bigger picture
Comparison table: Validation AUC

|    bootstrap_type    |                        |      grow_policy      |                    | 
|:---------------------|:-----------------------|:----------------------|:-------------------|
|                      |      SymmetricTree     |       Depthwise       |      Lossguide     | 
|       Bayesian       |         0.88646        |        0.86092        |       0.86015      | 
|                      |      SymmetricTree     |       Depthwise       |      Lossguide     |  
|      Bernoulli       |         0.88751        |        0.86133        |       0.86070      |  


- Comparison is in relative to alternative choices for each hyperparameter. 
- The configuration: boostrap_type = `Bernoulli`, grow_policy = `SymmetricTree` is used


In [81]:
# parameters specification
params = {
    'iterations': 1000,
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
#     'learning_rate': 0.03,
    'random_seed': 999,
    'l2_leaf_reg': 3.0,
    'bootstrap_type': 'Bernoulli',
#     'use_best_model': True,
    'random_strength': 1,
#     'depth': 6,
    'grow_policy': 'SymmetricTree',
#     'ignored_features': ignored_features, # not used due to inferior performance
    'boosting_type': 'Plain',
    'od_type': 'Iter',
    'od_wait': 50,
    'thread_count': -1,
    'logging_level': 'Verbose',
}

# prepare data pools
train_pool = Pool(X_train, y_train, cat_features=var_idx_cat)
validate_pool = Pool(X_val, y_val, cat_features=var_idx_cat)

# model specification
model = CatBoostClassifier(**params)

In [82]:
# fit the model
model.fit(train_pool, eval_set=validate_pool)

0:	test: 0.8076692	best: 0.8076692 (0)	total: 227ms	remaining: 3m 46s
1:	test: 0.8149584	best: 0.8149584 (1)	total: 449ms	remaining: 3m 44s
2:	test: 0.8282331	best: 0.8282331 (2)	total: 732ms	remaining: 4m 3s
3:	test: 0.8307306	best: 0.8307306 (3)	total: 979ms	remaining: 4m 3s
4:	test: 0.8301855	best: 0.8307306 (3)	total: 1.23s	remaining: 4m 4s
5:	test: 0.8307219	best: 0.8307306 (3)	total: 1.43s	remaining: 3m 56s
6:	test: 0.8313984	best: 0.8313984 (6)	total: 1.68s	remaining: 3m 58s
7:	test: 0.8323204	best: 0.8323204 (7)	total: 1.92s	remaining: 3m 58s
8:	test: 0.8316109	best: 0.8323204 (7)	total: 2.13s	remaining: 3m 54s
9:	test: 0.8325240	best: 0.8325240 (9)	total: 2.33s	remaining: 3m 51s
10:	test: 0.8322355	best: 0.8325240 (9)	total: 2.46s	remaining: 3m 41s
11:	test: 0.8322313	best: 0.8325240 (9)	total: 2.7s	remaining: 3m 42s
12:	test: 0.8320667	best: 0.8325240 (9)	total: 2.88s	remaining: 3m 38s
13:	test: 0.8332957	best: 0.8332957 (13)	total: 3.09s	remaining: 3m 37s
14:	test: 0.8335873

115:	test: 0.8497976	best: 0.8497976 (115)	total: 38.5s	remaining: 4m 53s
116:	test: 0.8503962	best: 0.8503962 (116)	total: 38.9s	remaining: 4m 53s
117:	test: 0.8504428	best: 0.8504428 (117)	total: 39.3s	remaining: 4m 53s
118:	test: 0.8505790	best: 0.8505790 (118)	total: 39.7s	remaining: 4m 53s
119:	test: 0.8507120	best: 0.8507120 (119)	total: 40.1s	remaining: 4m 53s
120:	test: 0.8508620	best: 0.8508620 (120)	total: 40.4s	remaining: 4m 53s
121:	test: 0.8509306	best: 0.8509306 (121)	total: 40.7s	remaining: 4m 53s
122:	test: 0.8509820	best: 0.8509820 (122)	total: 41.1s	remaining: 4m 53s
123:	test: 0.8510341	best: 0.8510341 (123)	total: 41.5s	remaining: 4m 52s
124:	test: 0.8510910	best: 0.8510910 (124)	total: 41.8s	remaining: 4m 52s
125:	test: 0.8511767	best: 0.8511767 (125)	total: 42.2s	remaining: 4m 52s
126:	test: 0.8512267	best: 0.8512267 (126)	total: 42.6s	remaining: 4m 52s
127:	test: 0.8512608	best: 0.8512608 (127)	total: 43s	remaining: 4m 52s
128:	test: 0.8512888	best: 0.8512888 (12

226:	test: 0.8585880	best: 0.8585880 (226)	total: 1m 19s	remaining: 4m 29s
227:	test: 0.8586413	best: 0.8586413 (227)	total: 1m 19s	remaining: 4m 29s
228:	test: 0.8586791	best: 0.8586791 (228)	total: 1m 19s	remaining: 4m 29s
229:	test: 0.8586926	best: 0.8586926 (229)	total: 1m 20s	remaining: 4m 28s
230:	test: 0.8587452	best: 0.8587452 (230)	total: 1m 20s	remaining: 4m 28s
231:	test: 0.8588119	best: 0.8588119 (231)	total: 1m 20s	remaining: 4m 28s
232:	test: 0.8588300	best: 0.8588300 (232)	total: 1m 21s	remaining: 4m 27s
233:	test: 0.8588365	best: 0.8588365 (233)	total: 1m 21s	remaining: 4m 27s
234:	test: 0.8588607	best: 0.8588607 (234)	total: 1m 22s	remaining: 4m 26s
235:	test: 0.8588913	best: 0.8588913 (235)	total: 1m 22s	remaining: 4m 26s
236:	test: 0.8589248	best: 0.8589248 (236)	total: 1m 22s	remaining: 4m 26s
237:	test: 0.8590051	best: 0.8590051 (237)	total: 1m 23s	remaining: 4m 25s
238:	test: 0.8590130	best: 0.8590130 (238)	total: 1m 23s	remaining: 4m 25s
239:	test: 0.8590411	best

336:	test: 0.8625147	best: 0.8625147 (336)	total: 1m 58s	remaining: 3m 53s
337:	test: 0.8625512	best: 0.8625512 (337)	total: 1m 59s	remaining: 3m 53s
338:	test: 0.8625654	best: 0.8625654 (338)	total: 1m 59s	remaining: 3m 53s
339:	test: 0.8625884	best: 0.8625884 (339)	total: 1m 59s	remaining: 3m 52s
340:	test: 0.8626016	best: 0.8626016 (340)	total: 2m	remaining: 3m 52s
341:	test: 0.8626213	best: 0.8626213 (341)	total: 2m	remaining: 3m 52s
342:	test: 0.8630141	best: 0.8630141 (342)	total: 2m 1s	remaining: 3m 51s
343:	test: 0.8630515	best: 0.8630515 (343)	total: 2m 1s	remaining: 3m 51s
344:	test: 0.8630796	best: 0.8630796 (344)	total: 2m 1s	remaining: 3m 50s
345:	test: 0.8630889	best: 0.8630889 (345)	total: 2m 1s	remaining: 3m 50s
346:	test: 0.8631131	best: 0.8631131 (346)	total: 2m 2s	remaining: 3m 50s
347:	test: 0.8631191	best: 0.8631191 (347)	total: 2m 2s	remaining: 3m 49s
348:	test: 0.8631273	best: 0.8631273 (348)	total: 2m 3s	remaining: 3m 49s
349:	test: 0.8631405	best: 0.8631405 (34

446:	test: 0.8699227	best: 0.8699227 (446)	total: 2m 38s	remaining: 3m 16s
447:	test: 0.8699413	best: 0.8699413 (447)	total: 2m 38s	remaining: 3m 15s
448:	test: 0.8699709	best: 0.8699709 (448)	total: 2m 39s	remaining: 3m 15s
449:	test: 0.8699905	best: 0.8699905 (449)	total: 2m 39s	remaining: 3m 15s
450:	test: 0.8705237	best: 0.8705237 (450)	total: 2m 39s	remaining: 3m 14s
451:	test: 0.8705348	best: 0.8705348 (451)	total: 2m 40s	remaining: 3m 14s
452:	test: 0.8705709	best: 0.8705709 (452)	total: 2m 40s	remaining: 3m 14s
453:	test: 0.8705872	best: 0.8705872 (453)	total: 2m 41s	remaining: 3m 13s
454:	test: 0.8706023	best: 0.8706023 (454)	total: 2m 41s	remaining: 3m 13s
455:	test: 0.8710045	best: 0.8710045 (455)	total: 2m 41s	remaining: 3m 13s
456:	test: 0.8710334	best: 0.8710334 (456)	total: 2m 42s	remaining: 3m 12s
457:	test: 0.8710447	best: 0.8710447 (457)	total: 2m 42s	remaining: 3m 12s
458:	test: 0.8710576	best: 0.8710576 (458)	total: 2m 42s	remaining: 3m 12s
459:	test: 0.8710867	best

557:	test: 0.8765738	best: 0.8765738 (557)	total: 3m 19s	remaining: 2m 37s
558:	test: 0.8765789	best: 0.8765789 (558)	total: 3m 19s	remaining: 2m 37s
559:	test: 0.8765887	best: 0.8765887 (559)	total: 3m 20s	remaining: 2m 37s
560:	test: 0.8766035	best: 0.8766035 (560)	total: 3m 20s	remaining: 2m 36s
561:	test: 0.8766161	best: 0.8766161 (561)	total: 3m 20s	remaining: 2m 36s
562:	test: 0.8766308	best: 0.8766308 (562)	total: 3m 21s	remaining: 2m 36s
563:	test: 0.8766469	best: 0.8766469 (563)	total: 3m 21s	remaining: 2m 35s
564:	test: 0.8766990	best: 0.8766990 (564)	total: 3m 21s	remaining: 2m 35s
565:	test: 0.8767089	best: 0.8767089 (565)	total: 3m 22s	remaining: 2m 34s
566:	test: 0.8767328	best: 0.8767328 (566)	total: 3m 22s	remaining: 2m 34s
567:	test: 0.8767347	best: 0.8767347 (567)	total: 3m 22s	remaining: 2m 34s
568:	test: 0.8767477	best: 0.8767477 (568)	total: 3m 23s	remaining: 2m 33s
569:	test: 0.8767575	best: 0.8767575 (569)	total: 3m 23s	remaining: 2m 33s
570:	test: 0.8767679	best

667:	test: 0.8813784	best: 0.8813784 (667)	total: 3m 58s	remaining: 1m 58s
668:	test: 0.8813903	best: 0.8813903 (668)	total: 3m 58s	remaining: 1m 58s
669:	test: 0.8813999	best: 0.8813999 (669)	total: 3m 59s	remaining: 1m 57s
670:	test: 0.8814061	best: 0.8814061 (670)	total: 3m 59s	remaining: 1m 57s
671:	test: 0.8814237	best: 0.8814237 (671)	total: 4m	remaining: 1m 57s
672:	test: 0.8814301	best: 0.8814301 (672)	total: 4m	remaining: 1m 56s
673:	test: 0.8814351	best: 0.8814351 (673)	total: 4m	remaining: 1m 56s
674:	test: 0.8814552	best: 0.8814552 (674)	total: 4m 1s	remaining: 1m 56s
675:	test: 0.8814669	best: 0.8814669 (675)	total: 4m 1s	remaining: 1m 55s
676:	test: 0.8816228	best: 0.8816228 (676)	total: 4m 1s	remaining: 1m 55s
677:	test: 0.8816227	best: 0.8816228 (676)	total: 4m 1s	remaining: 1m 54s
678:	test: 0.8818933	best: 0.8818933 (678)	total: 4m 2s	remaining: 1m 54s
679:	test: 0.8819072	best: 0.8819072 (679)	total: 4m 2s	remaining: 1m 54s
680:	test: 0.8819119	best: 0.8819119 (680)	

777:	test: 0.8843368	best: 0.8843412 (776)	total: 4m 37s	remaining: 1m 19s
778:	test: 0.8843470	best: 0.8843470 (778)	total: 4m 38s	remaining: 1m 18s
779:	test: 0.8843498	best: 0.8843498 (779)	total: 4m 38s	remaining: 1m 18s
780:	test: 0.8843504	best: 0.8843504 (780)	total: 4m 38s	remaining: 1m 18s
781:	test: 0.8843549	best: 0.8843549 (781)	total: 4m 39s	remaining: 1m 17s
782:	test: 0.8843634	best: 0.8843634 (782)	total: 4m 39s	remaining: 1m 17s
783:	test: 0.8843662	best: 0.8843662 (783)	total: 4m 39s	remaining: 1m 17s
784:	test: 0.8843708	best: 0.8843708 (784)	total: 4m 40s	remaining: 1m 16s
785:	test: 0.8843756	best: 0.8843756 (785)	total: 4m 40s	remaining: 1m 16s
786:	test: 0.8843932	best: 0.8843932 (786)	total: 4m 40s	remaining: 1m 16s
787:	test: 0.8844009	best: 0.8844009 (787)	total: 4m 41s	remaining: 1m 15s
788:	test: 0.8844055	best: 0.8844055 (788)	total: 4m 41s	remaining: 1m 15s
789:	test: 0.8844135	best: 0.8844135 (789)	total: 4m 41s	remaining: 1m 14s
790:	test: 0.8844209	best

889:	test: 0.8861095	best: 0.8861095 (889)	total: 5m 17s	remaining: 39.3s
890:	test: 0.8861100	best: 0.8861100 (890)	total: 5m 18s	remaining: 38.9s
891:	test: 0.8861128	best: 0.8861128 (891)	total: 5m 18s	remaining: 38.6s
892:	test: 0.8862680	best: 0.8862680 (892)	total: 5m 18s	remaining: 38.2s
893:	test: 0.8862760	best: 0.8862760 (893)	total: 5m 19s	remaining: 37.8s
894:	test: 0.8862850	best: 0.8862850 (894)	total: 5m 19s	remaining: 37.5s
895:	test: 0.8863745	best: 0.8863745 (895)	total: 5m 19s	remaining: 37.1s
896:	test: 0.8863803	best: 0.8863803 (896)	total: 5m 20s	remaining: 36.8s
897:	test: 0.8863866	best: 0.8863866 (897)	total: 5m 20s	remaining: 36.4s
898:	test: 0.8863886	best: 0.8863886 (898)	total: 5m 21s	remaining: 36.1s
899:	test: 0.8863890	best: 0.8863890 (899)	total: 5m 21s	remaining: 35.7s
900:	test: 0.8864772	best: 0.8864772 (900)	total: 5m 21s	remaining: 35.4s
901:	test: 0.8865003	best: 0.8865003 (901)	total: 5m 22s	remaining: 35s
902:	test: 0.8865010	best: 0.8865010 (90

In [84]:
model.get_best_score()
# expected results:
# {'learn': {'Logloss': 0.4325745826949464},
#  'validation': {'Logloss': 0.4204577527452197, 'AUC': 0.8875108831620899}}

{'learn': {'Logloss': 0.4325745826949464},
 'validation': {'Logloss': 0.4204577527452197, 'AUC': 0.8875108831620899}}

Exploration: Set a larger learning rate to speed up the training to allow the early stopping to perform.
This is because we want to see what train/val losses look like when the model stopped converging to design the penalised loss function.

In [139]:
# parameters specification
params = {
    'iterations': 10000,
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'learning_rate': 0.1,
    'random_seed': 999,
    'l2_leaf_reg': 3.0,
    'bootstrap_type': 'Bernoulli',
#     'use_best_model': True,
    'random_strength': 1,
#     'depth': 6,
    'grow_policy': 'SymmetricTree',
#     'ignored_features': ignored_features, # not used due to inferior performance
    'boosting_type': 'Plain',
    'od_type': 'Iter',
    'od_wait': 50,
    'thread_count': -1,
    'logging_level': 'Verbose',
}

# prepare data pools
train_pool = Pool(X_train, y_train, cat_features=var_idx_cat)
validate_pool = Pool(X_val, y_val, cat_features=var_idx_cat)

# model specification
model = CatBoostClassifier(**params)

# fit the model
model.fit(train_pool, eval_set=validate_pool)

0:	test: 0.8076692	best: 0.8076692 (0)	total: 226ms	remaining: 37m 39s
1:	test: 0.8219542	best: 0.8219542 (1)	total: 489ms	remaining: 40m 45s
2:	test: 0.8261418	best: 0.8261418 (2)	total: 707ms	remaining: 39m 17s
3:	test: 0.8260780	best: 0.8261418 (2)	total: 904ms	remaining: 37m 38s
4:	test: 0.8271170	best: 0.8271170 (4)	total: 1.21s	remaining: 40m 16s
5:	test: 0.8272522	best: 0.8272522 (5)	total: 1.47s	remaining: 40m 56s
6:	test: 0.8350041	best: 0.8350041 (6)	total: 1.85s	remaining: 44m 2s
7:	test: 0.8374285	best: 0.8374285 (7)	total: 2.08s	remaining: 43m 23s
8:	test: 0.8374979	best: 0.8374979 (8)	total: 2.33s	remaining: 43m 4s
9:	test: 0.8381641	best: 0.8381641 (9)	total: 2.71s	remaining: 45m 11s
10:	test: 0.8385925	best: 0.8385925 (10)	total: 2.96s	remaining: 44m 45s
11:	test: 0.8396001	best: 0.8396001 (11)	total: 3.33s	remaining: 46m 9s
12:	test: 0.8400136	best: 0.8400136 (12)	total: 3.64s	remaining: 46m 35s
13:	test: 0.8403176	best: 0.8403176 (13)	total: 4s	remaining: 47m 31s
14:	

113:	test: 0.8715565	best: 0.8715565 (113)	total: 39.5s	remaining: 57m 2s
114:	test: 0.8716198	best: 0.8716198 (114)	total: 39.8s	remaining: 56m 59s
115:	test: 0.8716526	best: 0.8716526 (115)	total: 40.1s	remaining: 56m 58s
116:	test: 0.8717632	best: 0.8717632 (116)	total: 40.5s	remaining: 57m
117:	test: 0.8717903	best: 0.8717903 (117)	total: 40.9s	remaining: 57m 1s
118:	test: 0.8718467	best: 0.8718467 (118)	total: 41.2s	remaining: 57m 1s
119:	test: 0.8718936	best: 0.8718936 (119)	total: 41.6s	remaining: 57m 2s
120:	test: 0.8719517	best: 0.8719517 (120)	total: 41.9s	remaining: 57m 3s
121:	test: 0.8719842	best: 0.8719842 (121)	total: 42.3s	remaining: 57m 4s
122:	test: 0.8720860	best: 0.8720860 (122)	total: 42.7s	remaining: 57m 6s
123:	test: 0.8721634	best: 0.8721634 (123)	total: 43s	remaining: 57m 3s
124:	test: 0.8729548	best: 0.8729548 (124)	total: 43.4s	remaining: 57m 8s
125:	test: 0.8730269	best: 0.8730269 (125)	total: 43.7s	remaining: 57m 6s
126:	test: 0.8731112	best: 0.8731112 (126

223:	test: 0.8832345	best: 0.8832345 (223)	total: 1m 18s	remaining: 57m 10s
224:	test: 0.8832517	best: 0.8832517 (224)	total: 1m 18s	remaining: 57m 10s
225:	test: 0.8832738	best: 0.8832738 (225)	total: 1m 19s	remaining: 57m 9s
226:	test: 0.8838650	best: 0.8838650 (226)	total: 1m 19s	remaining: 57m 9s
227:	test: 0.8839031	best: 0.8839031 (227)	total: 1m 20s	remaining: 57m 9s
228:	test: 0.8839359	best: 0.8839359 (228)	total: 1m 20s	remaining: 57m 10s
229:	test: 0.8839407	best: 0.8839407 (229)	total: 1m 20s	remaining: 57m 7s
230:	test: 0.8839948	best: 0.8839948 (230)	total: 1m 21s	remaining: 57m 6s
231:	test: 0.8840874	best: 0.8840874 (231)	total: 1m 21s	remaining: 57m 6s
232:	test: 0.8840909	best: 0.8840909 (232)	total: 1m 21s	remaining: 57m 5s
233:	test: 0.8843562	best: 0.8843562 (233)	total: 1m 22s	remaining: 57m 4s
234:	test: 0.8843810	best: 0.8843810 (234)	total: 1m 22s	remaining: 57m 6s
235:	test: 0.8843962	best: 0.8843962 (235)	total: 1m 22s	remaining: 57m 6s
236:	test: 0.8843976	b

332:	test: 0.8885059	best: 0.8885059 (332)	total: 1m 56s	remaining: 56m 27s
333:	test: 0.8885215	best: 0.8885215 (333)	total: 1m 57s	remaining: 56m 28s
334:	test: 0.8886456	best: 0.8886456 (334)	total: 1m 57s	remaining: 56m 27s
335:	test: 0.8886659	best: 0.8886659 (335)	total: 1m 57s	remaining: 56m 28s
336:	test: 0.8886711	best: 0.8886711 (336)	total: 1m 58s	remaining: 56m 28s
337:	test: 0.8886867	best: 0.8886867 (337)	total: 1m 58s	remaining: 56m 28s
338:	test: 0.8886863	best: 0.8886867 (337)	total: 1m 58s	remaining: 56m 27s
339:	test: 0.8886918	best: 0.8886918 (339)	total: 1m 59s	remaining: 56m 28s
340:	test: 0.8887052	best: 0.8887052 (340)	total: 1m 59s	remaining: 56m 25s
341:	test: 0.8887035	best: 0.8887052 (340)	total: 1m 59s	remaining: 56m 25s
342:	test: 0.8887057	best: 0.8887057 (342)	total: 2m	remaining: 56m 25s
343:	test: 0.8887146	best: 0.8887146 (343)	total: 2m	remaining: 56m 25s
344:	test: 0.8887233	best: 0.8887233 (344)	total: 2m	remaining: 56m 25s
345:	test: 0.8891492	bes

442:	test: 0.8923825	best: 0.8923825 (442)	total: 2m 35s	remaining: 56m
443:	test: 0.8923901	best: 0.8923901 (443)	total: 2m 36s	remaining: 55m 59s
444:	test: 0.8923839	best: 0.8923901 (443)	total: 2m 36s	remaining: 55m 59s
445:	test: 0.8923915	best: 0.8923915 (445)	total: 2m 36s	remaining: 55m 59s
446:	test: 0.8924015	best: 0.8924015 (446)	total: 2m 37s	remaining: 55m 59s
447:	test: 0.8924066	best: 0.8924066 (447)	total: 2m 37s	remaining: 55m 58s
448:	test: 0.8924040	best: 0.8924066 (447)	total: 2m 37s	remaining: 55m 56s
449:	test: 0.8924001	best: 0.8924066 (447)	total: 2m 38s	remaining: 55m 55s
450:	test: 0.8924080	best: 0.8924080 (450)	total: 2m 38s	remaining: 55m 54s
451:	test: 0.8924085	best: 0.8924085 (451)	total: 2m 38s	remaining: 55m 54s
452:	test: 0.8924086	best: 0.8924086 (452)	total: 2m 39s	remaining: 55m 53s
453:	test: 0.8928170	best: 0.8928170 (453)	total: 2m 39s	remaining: 55m 53s
454:	test: 0.8928300	best: 0.8928300 (454)	total: 2m 39s	remaining: 55m 51s
455:	test: 0.892

...

1833:	test: 0.9026968	best: 0.9026968 (1833)	total: 10m 48s	remaining: 48m 6s
1834:	test: 0.9026960	best: 0.9026968 (1833)	total: 10m 48s	remaining: 48m 5s
1835:	test: 0.9026981	best: 0.9026981 (1835)	total: 10m 49s	remaining: 48m 5s
1836:	test: 0.9026904	best: 0.9026981 (1835)	total: 10m 49s	remaining: 48m 5s
1837:	test: 0.9026875	best: 0.9026981 (1835)	total: 10m 49s	remaining: 48m 5s
1838:	test: 0.9026887	best: 0.9026981 (1835)	total: 10m 50s	remaining: 48m 4s
1839:	test: 0.9026889	best: 0.9026981 (1835)	total: 10m 50s	remaining: 48m 4s
1840:	test: 0.9026942	best: 0.9026981 (1835)	total: 10m 50s	remaining: 48m 4s
1841:	test: 0.9026928	best: 0.9026981 (1835)	total: 10m 51s	remaining: 48m 3s
1842:	test: 0.9026917	best: 0.9026981 (1835)	total: 10m 51s	remaining: 48m 2s
1843:	test: 0.9026973	best: 0.9026981 (1835)	total: 10m 51s	remaining: 48m 2s
1844:	test: 0.9026932	best: 0.9026981 (1835)	total: 10m 52s	remaining: 48m 2s
1845:	test: 0.9026860	best: 0.9026981 (1835)	total: 10m 52s	rema

1938:	test: 0.9029743	best: 0.9030065 (1933)	total: 11m 25s	remaining: 47m 31s
1939:	test: 0.9029696	best: 0.9030065 (1933)	total: 11m 26s	remaining: 47m 31s
1940:	test: 0.9029735	best: 0.9030065 (1933)	total: 11m 26s	remaining: 47m 30s
1941:	test: 0.9029582	best: 0.9030065 (1933)	total: 11m 27s	remaining: 47m 30s
1942:	test: 0.9029564	best: 0.9030065 (1933)	total: 11m 27s	remaining: 47m 30s
1943:	test: 0.9029575	best: 0.9030065 (1933)	total: 11m 27s	remaining: 47m 30s
1944:	test: 0.9029569	best: 0.9030065 (1933)	total: 11m 28s	remaining: 47m 29s
1945:	test: 0.9029565	best: 0.9030065 (1933)	total: 11m 28s	remaining: 47m 29s
1946:	test: 0.9029558	best: 0.9030065 (1933)	total: 11m 28s	remaining: 47m 28s
1947:	test: 0.9029547	best: 0.9030065 (1933)	total: 11m 29s	remaining: 47m 28s
1948:	test: 0.9029492	best: 0.9030065 (1933)	total: 11m 29s	remaining: 47m 28s
1949:	test: 0.9029622	best: 0.9030065 (1933)	total: 11m 29s	remaining: 47m 27s
1950:	test: 0.9029600	best: 0.9030065 (1933)	total: 

2043:	test: 0.9032287	best: 0.9032574 (2035)	total: 12m 4s	remaining: 46m 58s
2044:	test: 0.9032410	best: 0.9032574 (2035)	total: 12m 4s	remaining: 46m 58s
2045:	test: 0.9032427	best: 0.9032574 (2035)	total: 12m 4s	remaining: 46m 57s
2046:	test: 0.9032409	best: 0.9032574 (2035)	total: 12m 5s	remaining: 46m 57s
2047:	test: 0.9032294	best: 0.9032574 (2035)	total: 12m 5s	remaining: 46m 57s
2048:	test: 0.9032288	best: 0.9032574 (2035)	total: 12m 5s	remaining: 46m 56s
2049:	test: 0.9032278	best: 0.9032574 (2035)	total: 12m 6s	remaining: 46m 56s
2050:	test: 0.9032341	best: 0.9032574 (2035)	total: 12m 6s	remaining: 46m 56s
2051:	test: 0.9032356	best: 0.9032574 (2035)	total: 12m 6s	remaining: 46m 55s
2052:	test: 0.9032433	best: 0.9032574 (2035)	total: 12m 7s	remaining: 46m 55s
2053:	test: 0.9032450	best: 0.9032574 (2035)	total: 12m 7s	remaining: 46m 55s
2054:	test: 0.9032419	best: 0.9032574 (2035)	total: 12m 8s	remaining: 46m 54s
2055:	test: 0.9032334	best: 0.9032574 (2035)	total: 12m 8s	remai

2147:	test: 0.9036131	best: 0.9036259 (2137)	total: 12m 41s	remaining: 46m 25s
2148:	test: 0.9036166	best: 0.9036259 (2137)	total: 12m 42s	remaining: 46m 24s
2149:	test: 0.9036184	best: 0.9036259 (2137)	total: 12m 42s	remaining: 46m 24s
2150:	test: 0.9036114	best: 0.9036259 (2137)	total: 12m 42s	remaining: 46m 23s
2151:	test: 0.9036122	best: 0.9036259 (2137)	total: 12m 43s	remaining: 46m 23s
2152:	test: 0.9036084	best: 0.9036259 (2137)	total: 12m 43s	remaining: 46m 23s
2153:	test: 0.9036078	best: 0.9036259 (2137)	total: 12m 44s	remaining: 46m 23s
2154:	test: 0.9036096	best: 0.9036259 (2137)	total: 12m 44s	remaining: 46m 22s
2155:	test: 0.9036035	best: 0.9036259 (2137)	total: 12m 44s	remaining: 46m 22s
2156:	test: 0.9036002	best: 0.9036259 (2137)	total: 12m 45s	remaining: 46m 22s
2157:	test: 0.9035969	best: 0.9036259 (2137)	total: 12m 45s	remaining: 46m 21s
2158:	test: 0.9036036	best: 0.9036259 (2137)	total: 12m 45s	remaining: 46m 21s
2159:	test: 0.9036189	best: 0.9036259 (2137)	total: 

2252:	test: 0.9037796	best: 0.9038581 (2221)	total: 13m 20s	remaining: 45m 53s
2253:	test: 0.9037805	best: 0.9038581 (2221)	total: 13m 20s	remaining: 45m 52s
2254:	test: 0.9037844	best: 0.9038581 (2221)	total: 13m 21s	remaining: 45m 52s
2255:	test: 0.9037786	best: 0.9038581 (2221)	total: 13m 21s	remaining: 45m 52s
2256:	test: 0.9037769	best: 0.9038581 (2221)	total: 13m 22s	remaining: 45m 51s
2257:	test: 0.9037832	best: 0.9038581 (2221)	total: 13m 22s	remaining: 45m 51s
2258:	test: 0.9037925	best: 0.9038581 (2221)	total: 13m 23s	remaining: 45m 51s
2259:	test: 0.9037915	best: 0.9038581 (2221)	total: 13m 23s	remaining: 45m 51s
2260:	test: 0.9037964	best: 0.9038581 (2221)	total: 13m 23s	remaining: 45m 51s
2261:	test: 0.9037977	best: 0.9038581 (2221)	total: 13m 24s	remaining: 45m 51s
2262:	test: 0.9038009	best: 0.9038581 (2221)	total: 13m 24s	remaining: 45m 50s
2263:	test: 0.9037995	best: 0.9038581 (2221)	total: 13m 24s	remaining: 45m 50s
2264:	test: 0.9037985	best: 0.9038581 (2221)	total: 

In [141]:
model.get_best_score()
# expected results:
# {'learn': {'Logloss': 0.3805862397007077},
#  'validation': {'Logloss': 0.39049957949495656, 'AUC': 0.9038580769598226}}

{'learn': {'Logloss': 0.3805862397007077},
 'validation': {'Logloss': 0.39049957949495656, 'AUC': 0.9038580769598226}}

#### Extensive hyperparameter tuning (through hyperopt)
The hyperparameter setup is quite similar to what we have used in the best model above except that depth of the tree is not set automatically, learning rate is set to be 0.5 rather than the default with iterations set to 1000. This setting may not give the optimal hyperparameter settings; however, we cannot afford to train a thorough search. Therefore, a learning rate of 0.5 would allow the model to converge close to the minima within much shorter amount of time (i.e., gotten stopped by the overfitting detector - 20 iter). We considered it to be better than training a smaller number of iterations with lower learning rate (i.e., default setting) that would not converge to the minima.

In [42]:
# setting up parameter for the model

# define an objective function
def hyperopt_objective(params):
    # model specification
    model = CatBoostClassifier(iterations=1000,
                               loss_function='Logloss',
                               eval_metric='AUC',
                               learning_rate=0.5,
                               random_seed=999,
                               l2_leaf_reg=params['l2_leaf_reg'],
                               bootstrap_type='Bernoulli',
                               random_strength=params['random_strength'],
                               depth=params['depth'],
                               grow_policy='SymmetricTree',
                               boosting_type='Plain',
                               od_type='Iter',
                               od_wait=20,
                               thread_count=-1,
                               logging_level='Verbose')
    
    # prepare data pools
    # -- since we are doing cross-validation, design set is used
    design_pool = Pool(train_set[ind_idx_all], train_set[var_idx_response], cat_features=var_idx_cat)
    
    # perform cross-validation on the data set
    cv_data = cv(pool=design_pool,
                 params=model.get_params()
                )
    
    # get the index with best average AUC on the validation set
    idx_best_val = cv_data['test-AUC-mean'].idxmax()

    # get the log_loss correspond to the highest AUC
    # -- reminder that each step in the gradient descend is to find the minimum binary cross tropy
    # -- AUC is not a differentiable function
    val_logloss = cv_data.loc[idx_best_val, 'test-Logloss-mean']
    train_logloss = cv_data.loc[idx_best_val, 'train-Logloss-mean']
    
    # concerning the logloss/AUC in the above cell, AUC seems higher than we usually have in the test submission
    # we will try to penalise the logloss to reduce overfitting with following strategy:
    # -- penalise with coef=0.25 if the diff between train/val > 0.005; another 0.25 if diff between train/val > 0.01
    penalised_logloss = val_logloss \
    + 0.25 * np.maximum(0, val_logloss - train_logloss - 0.005) \
    + 0.25 * np.maximum(0, val_logloss - train_logloss - 0.01)
    
    return penalised_logloss

Concerning the computation cost, though we can tune for the `bootstrap_type`; to speed up the training process we decided to use `'Bernoulli'` method due to its speed advantage. Similary, we specify the `grow_policy` as `SymmetricTree` to optimise the training speed.

In [43]:
# specify a parameters space to tune via hyperopt  
params_space = { # comments show possible tuning range
    'l2_leaf_reg': hyperopt.hp.loguniform('l2_leaf_reg', 0, 5), # any positive value
#     'bootstrap_type': hyperopt.hp.choice('bootstrap_type', ['Bayesian', 'Bernoulli']), # the two choices
    'random_strength': hyperopt.hp.uniform('random_strength', 0, 50), # any value
    'depth': hyperopt.hp.quniform('depth', 0, 16, 1), # interger from [0, 16]
#     'grow_policy': hyperopt.hp.choice('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']), # the three choices
}

# initialise a trial object
trials = hyperopt.Trials()

# perform optimisation
best = hyperopt.fmin(hyperopt_objective,
                     space=params_space,
                     algo=hyperopt.tpe.suggest,
                     max_evals=10,
                     trials=trials,
                     rstate=np.random.RandomState(888)
)

print(best)

# expected results:
# best loss: 0.4246167908173705
# {'depth': 5.0, 'l2_leaf_reg': 41.7444539908069, 'random_strength': 41.134181677175654}

0:	test: 0.7966489	best: 0.7966489 (0)	total: 3.93s	remaining: 1h 5m 28s

1:	test: 0.8119184	best: 0.8119184 (1)	total: 6.78s	remaining: 56m 20s

2:	test: 0.8191269	best: 0.8191269 (2)	total: 8.47s	remaining: 46m 56s

3:	test: 0.8223703	best: 0.8223703 (3)	total: 11.1s	remaining: 46m 7s

4:	test: 0.8223179	best: 0.8223703 (3)                

5:	test: 0.8275742	best: 0.8275742 (5)                

6:	test: 0.8293523	best: 0.8293523 (6)                

7:	test: 0.8317550	best: 0.8317550 (7)                

8:	test: 0.8321304	best: 0.8321304 (8)	total: 20.9s	remaining: 38m 21s

9:	test: 0.8323485	best: 0.8323485 (9)	total: 24s	remaining: 39m 32s

10:	test: 0.8341372	best: 0.8341372 (10)              

11:	test: 0.8349285	best: 0.8349285 (11)              

12:	test: 0.8362651	best: 0.8362651 (12)              

13:	test: 0.8368637	best: 0.8368637 (13)              

14:	test: 0.8369821	best: 0.8369821 (14)              

15:	test: 0.8370348	best: 0.8370348 (15)              

16:	test:

106:	test: 0.8568932	best: 0.8568932 (106)                                         

107:	test: 0.8569979	best: 0.8569979 (107)	total: 28.7s	remaining: 3m 57s          

108:	test: 0.8570771	best: 0.8570771 (108)                                         

109:	test: 0.8571350	best: 0.8571350 (109)                                         

110:	test: 0.8572043	best: 0.8572043 (110)                                         

111:	test: 0.8572351	best: 0.8572351 (111)                                         

112:	test: 0.8572806	best: 0.8572806 (112)	total: 30s	remaining: 3m 55s            

113:	test: 0.8573177	best: 0.8573177 (113)                                         

114:	test: 0.8573810	best: 0.8573810 (114)                                         

115:	test: 0.8574133	best: 0.8574133 (115)                                         

116:	test: 0.8575001	best: 0.8575001 (116)	total: 31.1s	remaining: 3m 55s          

117:	test: 0.8575432	best: 0.8575432 (117)                       

298:	test: 0.8614832	best: 0.8614832 (298)                                         

299:	test: 0.8614821	best: 0.8614832 (298)	total: 1m 24s	remaining: 3m 16s         

300:	test: 0.8615058	best: 0.8615058 (300)                                         

301:	test: 0.8615028	best: 0.8615058 (300)                                         

302:	test: 0.8615040	best: 0.8615058 (300)	total: 1m 25s	remaining: 3m 15s         

303:	test: 0.8615301	best: 0.8615301 (303)                                         

304:	test: 0.8615690	best: 0.8615690 (304)                                         

305:	test: 0.8615836	best: 0.8615836 (305)                                         

306:	test: 0.8615833	best: 0.8615836 (305)	total: 1m 26s	remaining: 3m 14s         

307:	test: 0.8615927	best: 0.8615927 (307)                                         

308:	test: 0.8615982	best: 0.8615982 (308)                                         

309:	test: 0.8616052	best: 0.8616052 (309)                       

490:	test: 0.8628105	best: 0.8628105 (490)                                         

491:	test: 0.8628111	best: 0.8628111 (491)	total: 2m 17s	remaining: 2m 22s         

492:	test: 0.8628235	best: 0.8628235 (492)                                         

493:	test: 0.8628221	best: 0.8628235 (492)                                         

494:	test: 0.8628309	best: 0.8628309 (494)                                         

495:	test: 0.8628553	best: 0.8628553 (495)                                         

496:	test: 0.8628557	best: 0.8628557 (496)	total: 2m 18s	remaining: 2m 20s         

497:	test: 0.8628614	best: 0.8628614 (497)                                         

498:	test: 0.8628621	best: 0.8628621 (498)                                         

499:	test: 0.8628725	best: 0.8628725 (499)                                         

500:	test: 0.8628770	best: 0.8628770 (500)                                         

501:	test: 0.8628806	best: 0.8628806 (501)	total: 2m 20s	remainin

682:	test: 0.8635313	best: 0.8635313 (682)                                         

683:	test: 0.8635301	best: 0.8635313 (682)                                         

684:	test: 0.8635387	best: 0.8635387 (684)	total: 3m 9s	remaining: 1m 27s          

685:	test: 0.8635483	best: 0.8635483 (685)                                         

686:	test: 0.8635550	best: 0.8635550 (686)                                         

687:	test: 0.8635517	best: 0.8635550 (686)                                         

688:	test: 0.8635533	best: 0.8635550 (686)	total: 3m 10s	remaining: 1m 26s         

689:	test: 0.8635586	best: 0.8635586 (689)                                         

690:	test: 0.8635694	best: 0.8635694 (690)                                         

691:	test: 0.8635686	best: 0.8635694 (690)                                         

692:	test: 0.8635693	best: 0.8635694 (690)	total: 3m 12s	remaining: 1m 25s         

693:	test: 0.8635689	best: 0.8635694 (690)                       

...

139:	test: 0.8392788	best: 0.8392788 (139)	total: 14.1s	remaining: 1m 26s        

140:	test: 0.8392972	best: 0.8392972 (140)                                       

141:	test: 0.8393216	best: 0.8393216 (141)                                       

142:	test: 0.8393588	best: 0.8393588 (142)                                       

143:	test: 0.8393907	best: 0.8393907 (143)                                       

144:	test: 0.8394119	best: 0.8394119 (144)                                       

145:	test: 0.8394242	best: 0.8394242 (145)                                       

146:	test: 0.8394251	best: 0.8394251 (146)                                       

147:	test: 0.8394302	best: 0.8394302 (147)                                       

148:	test: 0.8394358	best: 0.8394358 (148)	total: 15s	remaining: 1m 25s          

149:	test: 0.8394534	best: 0.8394534 (149)                                       

150:	test: 0.8394842	best: 0.8394842 (150)                                       

151:

335:	test: 0.8408505	best: 0.8408505 (335)                                       

336:	test: 0.8408528	best: 0.8408528 (336)                                       

337:	test: 0.8408592	best: 0.8408592 (337)                                       

338:	test: 0.8408668	best: 0.8408668 (338)                                       

339:	test: 0.8408663	best: 0.8408668 (338)                                       

340:	test: 0.8408723	best: 0.8408723 (340)                                       

341:	test: 0.8408729	best: 0.8408729 (341)                                       

342:	test: 0.8408803	best: 0.8408803 (342)	total: 33.9s	remaining: 1m 4s         

343:	test: 0.8408817	best: 0.8408817 (343)                                       

344:	test: 0.8408817	best: 0.8408817 (343)                                       

345:	test: 0.8408907	best: 0.8408907 (345)                                       

346:	test: 0.8408959	best: 0.8408959 (346)                                       

347:

531:	test: 0.8412818	best: 0.8412845 (518)                                       

532:	test: 0.8412885	best: 0.8412885 (532)                                       

533:	test: 0.8412885	best: 0.8412885 (532)	total: 52.6s	remaining: 45.9s         

534:	test: 0.8412885	best: 0.8412885 (532)                                       

535:	test: 0.8412900	best: 0.8412900 (535)                                       

536:	test: 0.8412919	best: 0.8412919 (536)                                       

537:	test: 0.8412943	best: 0.8412943 (537)                                       

538:	test: 0.8412931	best: 0.8412943 (537)                                       

539:	test: 0.8413007	best: 0.8413007 (539)                                       

540:	test: 0.8413042	best: 0.8413042 (540)                                       

541:	test: 0.8413038	best: 0.8413042 (540)                                       

542:	test: 0.8413059	best: 0.8413059 (542)	total: 53.5s	remaining: 45s           

543:

727:	test: 0.8415327	best: 0.8415330 (725)                                       

728:	test: 0.8415335	best: 0.8415335 (728)                                       

729:	test: 0.8415343	best: 0.8415343 (729)	total: 1m 11s	remaining: 26.4s        

730:	test: 0.8415346	best: 0.8415346 (730)                                       

731:	test: 0.8415395	best: 0.8415395 (731)                                       

732:	test: 0.8415425	best: 0.8415425 (732)                                       

733:	test: 0.8415430	best: 0.8415430 (733)                                       

734:	test: 0.8415434	best: 0.8415434 (734)                                       

735:	test: 0.8415417	best: 0.8415434 (734)                                       

736:	test: 0.8415417	best: 0.8415434 (734)                                       

737:	test: 0.8415406	best: 0.8415434 (734)                                       

738:	test: 0.8415457	best: 0.8415457 (738)                                       

739:

111:	test: 0.8819792	best: 0.8819913 (106)	total: 2m 37s	remaining: 20m 50s      

112:	test: 0.8819632	best: 0.8819913 (106)	total: 2m 39s	remaining: 20m 50s      

113:	test: 0.8819599	best: 0.8819913 (106)	total: 2m 40s	remaining: 20m 50s      

114:	test: 0.8819884	best: 0.8819913 (106)	total: 2m 42s	remaining: 20m 50s      

115:	test: 0.8819803	best: 0.8819913 (106)	total: 2m 43s	remaining: 20m 48s      

116:	test: 0.8819934	best: 0.8819934 (116)	total: 2m 45s	remaining: 20m 47s      

117:	test: 0.8823324	best: 0.8823324 (117)	total: 2m 46s	remaining: 20m 45s      

118:	test: 0.8823288	best: 0.8823324 (117)	total: 2m 48s	remaining: 20m 44s      

119:	test: 0.8823352	best: 0.8823352 (119)	total: 2m 49s	remaining: 20m 44s      

120:	test: 0.8823299	best: 0.8823352 (119)	total: 2m 51s	remaining: 20m 44s      

121:	test: 0.8823019	best: 0.8823352 (119)	total: 2m 53s	remaining: 20m 45s      

122:	test: 0.8823259	best: 0.8823352 (119)	total: 2m 54s	remaining: 20m 44s      

123:

From the best tuning configuration, we noticed the CatBoost is really flexible and powerful. The tree depth is slightly lower than the default of 6. The L2 regularization (`l2_leaf_reg`) of 41.7 is much higher than the default suggested by the package. These parameters imply that the default configuration of CatBoost is prone to overfitting.

#### Make prediction on the test set

In [45]:
# setup hyperparameters
params = {
    'iterations': 1000,
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'random_seed': 888,
    'l2_leaf_reg': 41.7444539908069,
    'bootstrap_type': 'Bernoulli',
    'random_strength': 41.134181677175654,
    'depth': 5.0,
    'grow_policy': 'SymmetricTree',
    'boosting_type': 'Plain',
    'thread_count': -1,
    'logging_level': 'Verbose',
}

# prepare data pools
design_pool = Pool(train_set[ind_idx_all], train_set[var_idx_response], cat_features=var_idx_cat)

# model specification
best_model = CatBoostClassifier(**params)

# fit the model
best_model.fit(design_pool)

0:	total: 121ms	remaining: 2m 1s
1:	total: 446ms	remaining: 3m 42s
2:	total: 715ms	remaining: 3m 57s
3:	total: 1.01s	remaining: 4m 12s
4:	total: 1.37s	remaining: 4m 33s
5:	total: 1.71s	remaining: 4m 42s
6:	total: 2.04s	remaining: 4m 48s
7:	total: 2.32s	remaining: 4m 48s
8:	total: 2.66s	remaining: 4m 53s
9:	total: 2.76s	remaining: 4m 33s
10:	total: 2.99s	remaining: 4m 28s
11:	total: 3.29s	remaining: 4m 30s
12:	total: 3.51s	remaining: 4m 26s
13:	total: 3.66s	remaining: 4m 18s
14:	total: 3.95s	remaining: 4m 19s
15:	total: 4.19s	remaining: 4m 17s
16:	total: 4.49s	remaining: 4m 19s
17:	total: 4.52s	remaining: 4m 6s
18:	total: 4.82s	remaining: 4m 9s
19:	total: 4.92s	remaining: 4m 1s
20:	total: 5.21s	remaining: 4m 2s
21:	total: 5.45s	remaining: 4m 2s
22:	total: 5.77s	remaining: 4m 4s
23:	total: 6.09s	remaining: 4m 7s
24:	total: 6.42s	remaining: 4m 10s
25:	total: 6.68s	remaining: 4m 10s
26:	total: 6.92s	remaining: 4m 9s
27:	total: 7.07s	remaining: 4m 5s
28:	total: 7.4s	remaining: 4m 7s
29:	tot

233:	total: 57.3s	remaining: 3m 7s
234:	total: 57.6s	remaining: 3m 7s
235:	total: 57.7s	remaining: 3m 6s
236:	total: 57.8s	remaining: 3m 6s
237:	total: 57.9s	remaining: 3m 5s
238:	total: 58.2s	remaining: 3m 5s
239:	total: 58.5s	remaining: 3m 5s
240:	total: 58.6s	remaining: 3m 4s
241:	total: 58.8s	remaining: 3m 4s
242:	total: 58.8s	remaining: 3m 3s
243:	total: 58.9s	remaining: 3m 2s
244:	total: 59.2s	remaining: 3m 2s
245:	total: 59.4s	remaining: 3m 2s
246:	total: 59.7s	remaining: 3m 2s
247:	total: 1m	remaining: 3m 2s
248:	total: 1m	remaining: 3m 1s
249:	total: 1m	remaining: 3m 1s
250:	total: 1m	remaining: 3m 1s
251:	total: 1m 1s	remaining: 3m 1s
252:	total: 1m 1s	remaining: 3m 1s
253:	total: 1m 1s	remaining: 3m
254:	total: 1m 1s	remaining: 3m
255:	total: 1m 2s	remaining: 3m
256:	total: 1m 2s	remaining: 2m 59s
257:	total: 1m 2s	remaining: 2m 59s
258:	total: 1m 2s	remaining: 2m 59s
259:	total: 1m 2s	remaining: 2m 59s
260:	total: 1m 3s	remaining: 2m 59s
261:	total: 1m 3s	remaining: 2m 58s


458:	total: 1m 52s	remaining: 2m 12s
459:	total: 1m 52s	remaining: 2m 12s
460:	total: 1m 52s	remaining: 2m 11s
461:	total: 1m 52s	remaining: 2m 11s
462:	total: 1m 53s	remaining: 2m 11s
463:	total: 1m 53s	remaining: 2m 10s
464:	total: 1m 53s	remaining: 2m 10s
465:	total: 1m 53s	remaining: 2m 10s
466:	total: 1m 54s	remaining: 2m 10s
467:	total: 1m 54s	remaining: 2m 9s
468:	total: 1m 54s	remaining: 2m 9s
469:	total: 1m 54s	remaining: 2m 9s
470:	total: 1m 55s	remaining: 2m 9s
471:	total: 1m 55s	remaining: 2m 9s
472:	total: 1m 55s	remaining: 2m 8s
473:	total: 1m 55s	remaining: 2m 8s
474:	total: 1m 56s	remaining: 2m 8s
475:	total: 1m 56s	remaining: 2m 8s
476:	total: 1m 56s	remaining: 2m 7s
477:	total: 1m 56s	remaining: 2m 7s
478:	total: 1m 57s	remaining: 2m 7s
479:	total: 1m 57s	remaining: 2m 7s
480:	total: 1m 57s	remaining: 2m 6s
481:	total: 1m 57s	remaining: 2m 6s
482:	total: 1m 58s	remaining: 2m 6s
483:	total: 1m 58s	remaining: 2m 6s
484:	total: 1m 58s	remaining: 2m 6s
485:	total: 1m 59s	

683:	total: 3m 2s	remaining: 1m 24s
684:	total: 3m 2s	remaining: 1m 23s
685:	total: 3m 2s	remaining: 1m 23s
686:	total: 3m 3s	remaining: 1m 23s
687:	total: 3m 3s	remaining: 1m 23s
688:	total: 3m 3s	remaining: 1m 22s
689:	total: 3m 4s	remaining: 1m 22s
690:	total: 3m 4s	remaining: 1m 22s
691:	total: 3m 4s	remaining: 1m 22s
692:	total: 3m 5s	remaining: 1m 21s
693:	total: 3m 5s	remaining: 1m 21s
694:	total: 3m 5s	remaining: 1m 21s
695:	total: 3m 6s	remaining: 1m 21s
696:	total: 3m 6s	remaining: 1m 21s
697:	total: 3m 6s	remaining: 1m 20s
698:	total: 3m 6s	remaining: 1m 20s
699:	total: 3m 7s	remaining: 1m 20s
700:	total: 3m 7s	remaining: 1m 20s
701:	total: 3m 7s	remaining: 1m 19s
702:	total: 3m 8s	remaining: 1m 19s
703:	total: 3m 8s	remaining: 1m 19s
704:	total: 3m 8s	remaining: 1m 19s
705:	total: 3m 9s	remaining: 1m 18s
706:	total: 3m 9s	remaining: 1m 18s
707:	total: 3m 9s	remaining: 1m 18s
708:	total: 3m 10s	remaining: 1m 18s
709:	total: 3m 10s	remaining: 1m 17s
710:	total: 3m 10s	remaini

912:	total: 4m 15s	remaining: 24.3s
913:	total: 4m 15s	remaining: 24s
914:	total: 4m 15s	remaining: 23.8s
915:	total: 4m 16s	remaining: 23.5s
916:	total: 4m 16s	remaining: 23.2s
917:	total: 4m 16s	remaining: 22.9s
918:	total: 4m 17s	remaining: 22.7s
919:	total: 4m 17s	remaining: 22.4s
920:	total: 4m 17s	remaining: 22.1s
921:	total: 4m 18s	remaining: 21.8s
922:	total: 4m 18s	remaining: 21.6s
923:	total: 4m 18s	remaining: 21.3s
924:	total: 4m 18s	remaining: 21s
925:	total: 4m 19s	remaining: 20.7s
926:	total: 4m 19s	remaining: 20.4s
927:	total: 4m 19s	remaining: 20.2s
928:	total: 4m 20s	remaining: 19.9s
929:	total: 4m 20s	remaining: 19.6s
930:	total: 4m 20s	remaining: 19.3s
931:	total: 4m 21s	remaining: 19.1s
932:	total: 4m 21s	remaining: 18.8s
933:	total: 4m 21s	remaining: 18.5s
934:	total: 4m 22s	remaining: 18.2s
935:	total: 4m 22s	remaining: 17.9s
936:	total: 4m 22s	remaining: 17.7s
937:	total: 4m 23s	remaining: 17.4s
938:	total: 4m 23s	remaining: 17.1s
939:	total: 4m 23s	remaining: 16

In [46]:
# make prediction on the test set
model_pred_test = best_model.predict_proba(test_set[ind_idx_all])

#### Create submission file

In [48]:
# get the probability of response being 1
model_pred_test = pd.DataFrame(model_pred_test)[1]
# create submission file
submission = pd.DataFrame({
    'unique_id': test_set['unique_id'],
    'Predicted': model_pred_test
})
# save the submission file
f_name = 'Group_8_part4.csv'
f_path = os.path.join(dirPOutput, f_name)
submission.to_csv(f_path, index=False)

In [49]:
# preview the submission file
submission

,unique_id,Predicted
0,6,0.051090
1,16,0.829659
2,17,0.226710
3,18,0.598905
4,19,0.019363
...,...,...
296685,2265630,0.965119
296686,2265631,0.026760
296687,2265632,0.548777
296688,2265637,0.214398


### Note on Part 4

#### Kaggle score: 0.87107
<img src="../screenshots/Group_8_part4_screenshot.png">

For Part 4, the logical processes for our solution are as follows:
- We strategically cleaned missing values in a similar manner as previous parts
- Trained a default gradient boosting model through CatBoost to provide a baseline for comparison (Kaggle AUC = 0.87561)
- Experimented with the model by performing feature selection (provided by CatBoost package) before training (Kaggle AUC = 0.87099) -- due to inferior performance, it is not used
- Experimented with different hyperparameters choices: `bootstrap_type` and `grow_policy`	
- Performed hyperparameter tuning through hyperopt package
- Specified custom loss function to penalise the overfitting: penalised log loss
- Trained the best hyperparameter configurations on the design set
- Predicted the test set and created the submission file for Kaggle competition

From Part 4, we have got a chance to learn many things through experiments. First, we were curious about feature selection and the tree-based model as we learned that tree-based models are good at selecting important features. The experiment result was in line with our expectation that selecting features before training a gradient boosting model is not a good idea. Unfortunately, we did not manage to perform a thorough search through the hyperparameters due to the computational challenges. This problem set has introduced us to a bottleneck when trying to get the optimal model. Even though the hyperopt package provided parallel optimisation to speed up the hyperparameter tuning process, it still takes a lot of time to tune the model. We strongly believe that a thorough tuning would give a model that performs much better than the default configurations.

Nevertheless, a gradient boosting machine has given us much better result as compared to the penalised logistic regression models.

| Kaggle Competition   |  Part 2    |   Part 3    |    Part 4     |    Part 5    |
|:---------------------|:-----------|:------------|:--------------|--------------|
|         AUC          |  0.62644   |  0.84734    |   0.87107     |   0.83073    |

#### Not related to Part 4
We just want to try Bagging with default Catboost to experiment on how much score can be obtained by maintaining the signal while cancelled out noises through an average of multiple predictions. The train set uses this time is `1m` set.

Preprocessing is done in the same manner. We will now perform bagging, averaging 16 results.

In [25]:
# initialisation
seeds = [111, 222, 333, 444, 555, 666, 777, 888, 999, 1010, 1111, 2222, 3333, 4444, 5555, 6666]
df = pd.DataFrame()
for seed in seeds:
    # split data into train/validation sets
    X_train, X_val, y_train, y_val  = train_test_split(train_set[ind_idx_all], train_set[var_idx_response],
                                                       test_size=0.1, random_state=seed)
    # prepare parameter
    params = {
    'iterations': 1000,
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'random_seed': seed,
    'bootstrap_type': 'Bernoulli',
    'random_strength': seed,
    'grow_policy': 'SymmetricTree',
    'boosting_type': 'Plain',
    'od_type': 'Iter',
    'od_wait': 50,
    'thread_count': -1,
    'logging_level': 'Verbose',
    }
    
    # prepare data pools
    train_pool = Pool(X_train, y_train, cat_features=var_idx_cat)
    validate_pool = Pool(X_val, y_val, cat_features=var_idx_cat)

    # model specification
    model = CatBoostClassifier(**params)
    
    # fit the model
    model.fit(train_pool, eval_set=validate_pool)
    
    # make prediction on the test set
    model_pred_test = model.predict_proba(test_set[ind_idx_all])
    model_pred_test = pd.DataFrame(model_pred_test)[1]
    
    # add prediction to df
    df[seed] = model_pred_test

Learning rate set to 0.120753
0:	test: 0.5133250	best: 0.5133250 (0)	total: 167ms	remaining: 2m 47s
1:	test: 0.5654328	best: 0.5654328 (1)	total: 553ms	remaining: 4m 36s
2:	test: 0.5663172	best: 0.5663172 (2)	total: 690ms	remaining: 3m 49s
3:	test: 0.7749205	best: 0.7749205 (3)	total: 1.1s	remaining: 4m 34s
4:	test: 0.7771358	best: 0.7771358 (4)	total: 1.19s	remaining: 3m 55s
5:	test: 0.7771358	best: 0.7771358 (4)	total: 1.22s	remaining: 3m 21s
6:	test: 0.7769334	best: 0.7771358 (4)	total: 1.57s	remaining: 3m 42s
7:	test: 0.7753228	best: 0.7771358 (4)	total: 1.76s	remaining: 3m 38s
8:	test: 0.7851593	best: 0.7851593 (8)	total: 2.12s	remaining: 3m 52s
9:	test: 0.7856174	best: 0.7856174 (9)	total: 2.2s	remaining: 3m 37s
10:	test: 0.7855249	best: 0.7856174 (9)	total: 2.52s	remaining: 3m 47s
11:	test: 0.7900566	best: 0.7900566 (11)	total: 2.93s	remaining: 4m 1s
12:	test: 0.7901797	best: 0.7901797 (12)	total: 3.02s	remaining: 3m 49s
13:	test: 0.8152072	best: 0.8152072 (13)	total: 3.42s	rema

114:	test: 0.8338508	best: 0.8338508 (114)	total: 27.2s	remaining: 3m 29s
115:	test: 0.8340284	best: 0.8340284 (115)	total: 27.5s	remaining: 3m 29s
116:	test: 0.8340770	best: 0.8340770 (116)	total: 27.8s	remaining: 3m 30s
117:	test: 0.8354684	best: 0.8354684 (117)	total: 28.3s	remaining: 3m 31s
118:	test: 0.8360879	best: 0.8360879 (118)	total: 28.6s	remaining: 3m 31s
119:	test: 0.8362280	best: 0.8362280 (119)	total: 29s	remaining: 3m 32s
120:	test: 0.8366618	best: 0.8366618 (120)	total: 29.2s	remaining: 3m 32s
121:	test: 0.8367493	best: 0.8367493 (121)	total: 29.3s	remaining: 3m 31s
122:	test: 0.8368298	best: 0.8368298 (122)	total: 29.6s	remaining: 3m 30s
123:	test: 0.8367903	best: 0.8368298 (122)	total: 30s	remaining: 3m 31s
124:	test: 0.8378985	best: 0.8378985 (124)	total: 30.3s	remaining: 3m 32s
125:	test: 0.8395953	best: 0.8395953 (125)	total: 30.7s	remaining: 3m 32s
126:	test: 0.8396896	best: 0.8396896 (126)	total: 30.8s	remaining: 3m 31s
127:	test: 0.8398006	best: 0.8398006 (127)

225:	test: 0.8761679	best: 0.8761679 (225)	total: 1m 7s	remaining: 3m 50s
226:	test: 0.8762689	best: 0.8762689 (226)	total: 1m 7s	remaining: 3m 50s
227:	test: 0.8763123	best: 0.8763123 (227)	total: 1m 7s	remaining: 3m 50s
228:	test: 0.8763573	best: 0.8763573 (228)	total: 1m 8s	remaining: 3m 50s
229:	test: 0.8765587	best: 0.8765587 (229)	total: 1m 8s	remaining: 3m 49s
230:	test: 0.8765728	best: 0.8765728 (230)	total: 1m 9s	remaining: 3m 49s
231:	test: 0.8766343	best: 0.8766343 (231)	total: 1m 9s	remaining: 3m 49s
232:	test: 0.8779876	best: 0.8779876 (232)	total: 1m 9s	remaining: 3m 49s
233:	test: 0.8780423	best: 0.8780423 (233)	total: 1m 10s	remaining: 3m 49s
234:	test: 0.8780769	best: 0.8780769 (234)	total: 1m 10s	remaining: 3m 49s
235:	test: 0.8781433	best: 0.8781433 (235)	total: 1m 10s	remaining: 3m 49s
236:	test: 0.8782406	best: 0.8782406 (236)	total: 1m 11s	remaining: 3m 49s
237:	test: 0.8782590	best: 0.8782590 (237)	total: 1m 11s	remaining: 3m 49s
238:	test: 0.8782927	best: 0.8782

335:	test: 0.8852038	best: 0.8852038 (335)	total: 1m 47s	remaining: 3m 33s
336:	test: 0.8851969	best: 0.8852038 (335)	total: 1m 48s	remaining: 3m 32s
337:	test: 0.8852081	best: 0.8852081 (337)	total: 1m 48s	remaining: 3m 32s
338:	test: 0.8852157	best: 0.8852157 (338)	total: 1m 48s	remaining: 3m 32s
339:	test: 0.8852210	best: 0.8852210 (339)	total: 1m 49s	remaining: 3m 32s
340:	test: 0.8852517	best: 0.8852517 (340)	total: 1m 49s	remaining: 3m 31s
341:	test: 0.8852585	best: 0.8852585 (341)	total: 1m 49s	remaining: 3m 31s
342:	test: 0.8852593	best: 0.8852593 (342)	total: 1m 50s	remaining: 3m 31s
343:	test: 0.8862633	best: 0.8862633 (343)	total: 1m 50s	remaining: 3m 31s
344:	test: 0.8862621	best: 0.8862633 (343)	total: 1m 51s	remaining: 3m 30s
345:	test: 0.8862629	best: 0.8862633 (343)	total: 1m 51s	remaining: 3m 30s
346:	test: 0.8862668	best: 0.8862668 (346)	total: 1m 51s	remaining: 3m 30s
347:	test: 0.8862826	best: 0.8862826 (347)	total: 1m 52s	remaining: 3m 30s
348:	test: 0.8864158	best

445:	test: 0.8897135	best: 0.8897135 (445)	total: 2m 28s	remaining: 3m 4s
446:	test: 0.8897105	best: 0.8897135 (445)	total: 2m 28s	remaining: 3m 3s
447:	test: 0.8897481	best: 0.8897481 (447)	total: 2m 28s	remaining: 3m 3s
448:	test: 0.8897564	best: 0.8897564 (448)	total: 2m 29s	remaining: 3m 3s
449:	test: 0.8897540	best: 0.8897564 (448)	total: 2m 29s	remaining: 3m 2s
450:	test: 0.8897605	best: 0.8897605 (450)	total: 2m 29s	remaining: 3m 2s
451:	test: 0.8897577	best: 0.8897605 (450)	total: 2m 30s	remaining: 3m 2s
452:	test: 0.8897660	best: 0.8897660 (452)	total: 2m 30s	remaining: 3m 1s
453:	test: 0.8897708	best: 0.8897708 (453)	total: 2m 30s	remaining: 3m 1s
454:	test: 0.8897908	best: 0.8897908 (454)	total: 2m 31s	remaining: 3m 1s
455:	test: 0.8900897	best: 0.8900897 (455)	total: 2m 31s	remaining: 3m 1s
456:	test: 0.8900807	best: 0.8900897 (455)	total: 2m 32s	remaining: 3m
457:	test: 0.8900794	best: 0.8900897 (455)	total: 2m 32s	remaining: 3m
458:	test: 0.8900721	best: 0.8900897 (455)	t

...

678:	test: 0.8974775	best: 0.8974775 (678)	total: 3m 43s	remaining: 1m 45s
679:	test: 0.8974794	best: 0.8974794 (679)	total: 3m 43s	remaining: 1m 45s
680:	test: 0.8974908	best: 0.8974908 (680)	total: 3m 43s	remaining: 1m 44s
681:	test: 0.8975036	best: 0.8975036 (681)	total: 3m 44s	remaining: 1m 44s
682:	test: 0.8975094	best: 0.8975094 (682)	total: 3m 44s	remaining: 1m 44s
683:	test: 0.8975032	best: 0.8975094 (682)	total: 3m 45s	remaining: 1m 43s
684:	test: 0.8975070	best: 0.8975094 (682)	total: 3m 45s	remaining: 1m 43s
685:	test: 0.8975099	best: 0.8975099 (685)	total: 3m 45s	remaining: 1m 43s
686:	test: 0.8975955	best: 0.8975955 (686)	total: 3m 46s	remaining: 1m 42s
687:	test: 0.8976038	best: 0.8976038 (687)	total: 3m 46s	remaining: 1m 42s
688:	test: 0.8976839	best: 0.8976839 (688)	total: 3m 46s	remaining: 1m 42s
689:	test: 0.8976981	best: 0.8976981 (689)	total: 3m 47s	remaining: 1m 42s
690:	test: 0.8979460	best: 0.8979460 (690)	total: 3m 47s	remaining: 1m 41s
691:	test: 0.8979461	best

788:	test: 0.8986587	best: 0.8986587 (788)	total: 4m 22s	remaining: 1m 10s
789:	test: 0.8986595	best: 0.8986595 (789)	total: 4m 22s	remaining: 1m 9s
790:	test: 0.8986530	best: 0.8986595 (789)	total: 4m 23s	remaining: 1m 9s
791:	test: 0.8986547	best: 0.8986595 (789)	total: 4m 23s	remaining: 1m 9s
792:	test: 0.8986442	best: 0.8986595 (789)	total: 4m 23s	remaining: 1m 8s
793:	test: 0.8986537	best: 0.8986595 (789)	total: 4m 24s	remaining: 1m 8s
794:	test: 0.8986537	best: 0.8986595 (789)	total: 4m 24s	remaining: 1m 8s
795:	test: 0.8986570	best: 0.8986595 (789)	total: 4m 24s	remaining: 1m 7s
796:	test: 0.8986569	best: 0.8986595 (789)	total: 4m 25s	remaining: 1m 7s
797:	test: 0.8986667	best: 0.8986667 (797)	total: 4m 25s	remaining: 1m 7s
798:	test: 0.8986624	best: 0.8986667 (797)	total: 4m 25s	remaining: 1m 6s
799:	test: 0.8986655	best: 0.8986667 (797)	total: 4m 26s	remaining: 1m 6s
800:	test: 0.8986659	best: 0.8986667 (797)	total: 4m 26s	remaining: 1m 6s
801:	test: 0.8986692	best: 0.8986692 

900:	test: 0.9010804	best: 0.9010804 (900)	total: 5m 2s	remaining: 33.3s
901:	test: 0.9010883	best: 0.9010883 (901)	total: 5m 3s	remaining: 32.9s
902:	test: 0.9010854	best: 0.9010883 (901)	total: 5m 3s	remaining: 32.6s
903:	test: 0.9010751	best: 0.9010883 (901)	total: 5m 3s	remaining: 32.3s
904:	test: 0.9010661	best: 0.9010883 (901)	total: 5m 4s	remaining: 31.9s
905:	test: 0.9010648	best: 0.9010883 (901)	total: 5m 4s	remaining: 31.6s
906:	test: 0.9010623	best: 0.9010883 (901)	total: 5m 4s	remaining: 31.2s
907:	test: 0.9010616	best: 0.9010883 (901)	total: 5m 5s	remaining: 30.9s
908:	test: 0.9010550	best: 0.9010883 (901)	total: 5m 5s	remaining: 30.6s
909:	test: 0.9010485	best: 0.9010883 (901)	total: 5m 5s	remaining: 30.2s
910:	test: 0.9010526	best: 0.9010883 (901)	total: 5m 6s	remaining: 29.9s
911:	test: 0.9010522	best: 0.9010883 (901)	total: 5m 6s	remaining: 29.6s
912:	test: 0.9010699	best: 0.9010883 (901)	total: 5m 6s	remaining: 29.2s
913:	test: 0.9010653	best: 0.9010883 (901)	total: 5

12:	test: 0.6407492	best: 0.6407492 (12)	total: 3.08s	remaining: 3m 53s
13:	test: 0.6407492	best: 0.6407492 (12)	total: 3.12s	remaining: 3m 39s
14:	test: 0.6455318	best: 0.6455318 (14)	total: 3.26s	remaining: 3m 34s
15:	test: 0.6478429	best: 0.6478429 (15)	total: 3.57s	remaining: 3m 39s
16:	test: 0.6570284	best: 0.6570284 (16)	total: 3.77s	remaining: 3m 37s
17:	test: 0.7965299	best: 0.7965299 (17)	total: 4.09s	remaining: 3m 42s
18:	test: 0.7962934	best: 0.7965299 (17)	total: 4.36s	remaining: 3m 45s
19:	test: 0.7962276	best: 0.7965299 (17)	total: 4.71s	remaining: 3m 50s
20:	test: 0.7955408	best: 0.7965299 (17)	total: 4.86s	remaining: 3m 46s
21:	test: 0.7955408	best: 0.7965299 (17)	total: 4.94s	remaining: 3m 39s
22:	test: 0.7955408	best: 0.7965299 (17)	total: 4.98s	remaining: 3m 31s
23:	test: 0.7948960	best: 0.7965299 (17)	total: 5.35s	remaining: 3m 37s
24:	test: 0.7948960	best: 0.7965299 (17)	total: 5.5s	remaining: 3m 34s
25:	test: 0.7948826	best: 0.7965299 (17)	total: 5.69s	remaining: 

41:	test: 0.7340513	best: 0.7342173 (35)	total: 7.96s	remaining: 3m 1s
42:	test: 0.7346410	best: 0.7346410 (42)	total: 8.17s	remaining: 3m 1s
43:	test: 0.7334303	best: 0.7346410 (42)	total: 8.53s	remaining: 3m 5s
44:	test: 0.7334303	best: 0.7346410 (42)	total: 8.56s	remaining: 3m 1s
45:	test: 0.7331586	best: 0.7346410 (42)	total: 8.88s	remaining: 3m 4s
46:	test: 0.7803302	best: 0.7803302 (46)	total: 9.18s	remaining: 3m 6s
47:	test: 0.7805725	best: 0.7805725 (47)	total: 9.51s	remaining: 3m 8s
48:	test: 0.7805092	best: 0.7805725 (47)	total: 9.65s	remaining: 3m 7s
49:	test: 0.7824363	best: 0.7824363 (49)	total: 9.97s	remaining: 3m 9s
50:	test: 0.7825037	best: 0.7825037 (50)	total: 10.3s	remaining: 3m 10s
51:	test: 0.7825037	best: 0.7825037 (50)	total: 10.3s	remaining: 3m 7s
52:	test: 0.7825037	best: 0.7825037 (50)	total: 10.3s	remaining: 3m 4s
53:	test: 0.7824962	best: 0.7825037 (50)	total: 10.7s	remaining: 3m 6s
54:	test: 0.7825327	best: 0.7825327 (54)	total: 10.8s	remaining: 3m 4s
55:	t

In [29]:
# 7th,15th,16th models were not used due to stopping early
df = df.drop(columns=[777, 5555, 6666])

# get the mean
df['mean'] = df.mean(axis=1)

In [34]:
# create submission file
submission = pd.DataFrame({
    'unique_id': test_set['unique_id'],
    'Predicted': df['mean']
})
# save the submission file
f_name = 'ensemble.csv'
f_path = os.path.join(dirPOutput, f_name)
submission.to_csv(f_path, index=False)

In [35]:
submission

,unique_id,Predicted
0,6,0.058417
1,16,0.835440
2,17,0.329853
3,18,0.606910
4,19,0.011700
...,...,...
296685,2265630,0.973729
296686,2265631,0.019668
296687,2265632,0.515886
296688,2265637,0.270679


#### Kaggle score: 0.87715
<img src="../screenshots/ensemble.png">